In [1]:
from pyspark.sql import SparkSession
import os

# Inicializar Spark
spark = SparkSession.builder \
    .appName("ProyectoSparkie-Recomendaciones") \
    .master("local[*]") \
    .getOrCreate()

sc = spark.sparkContext

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
25/12/07 21:13:27 WARN Utils: Your hostname, maria-lopez-VirtualBox, resolves to a loopback address: 127.0.1.1; using 10.0.2.15 instead (on interface enp0s3)
25/12/07 21:13:27 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/12/07 21:13:28 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
# Cargar la matriz de similitudes del Paso 4
sim_path = "../data/processed/similarity_matrix_rdd"

# Leer el RDD guardado
rdd_similarities = sc.textFile(sim_path)

# Parsear el formato
def parse_similarity(line):
    """Convierte string del RDD a tupla Python"""
    import ast
    parsed = ast.literal_eval(line)
    doc1 = parsed[0]
    doc2 = parsed[1]
    similarity = parsed[2]
    return (doc1, doc2, similarity)

rdd_sim_parsed = rdd_similarities.map(parse_similarity)

print("Matriz de similitudes cargada")
print(f"   Total de pares: {rdd_sim_parsed.count()}")

Matriz de similitudes cargada


[Stage 0:>                                                          (0 + 2) / 3]

   Total de pares: 4851


In [3]:
# Extraer lista única de todos los libros
libros_disponibles = rdd_sim_parsed.flatMap(lambda x: [x[0], x[1]]).distinct().collect()

print(f"Total de libros disponibles: {len(libros_disponibles)}")

Total de libros disponibles: 99


In [4]:
def buscar_libro(nombre_libro, libros_disponibles):
    """
    Busca un libro en la lista.
    
    Parámetros:
    - nombre_libro: nombre del libro
    - libros_disponibles: lista de todos los libros
    
    Retorna:
    - nombre_completo del libro si existe
    - None si no existe
    """
    # Búsqueda exacta
    nombre_lower = nombre_libro.lower()
    
    for libro in libros_disponibles:
        if libro.lower() == nombre_lower:
            return libro
    
    # Búsqueda parcial
    coincidencias = [libro for libro in libros_disponibles 
                     if nombre_lower in libro.lower()]
    
    if len(coincidencias) == 1:
        print(f"Libro encontrado: {coincidencias[0]}")
        return coincidencias[0]
    elif len(coincidencias) > 1:
        print(f"Se encontraron {len(coincidencias)} libros que coinciden:")
        for i, libro in enumerate(coincidencias[:5], 1):
            print(f"   {i}. {libro}")
        if len(coincidencias) > 5:
            print(f"   ... y {len(coincidencias) - 5} más")
        print("\n Por favor, sea más específico con el nombre.")
        return None
    else:
        print(f"Error: El libro '{nombre_libro}' no existe en la colección.")
        print("\n Sugerencias (libros similares):")
        # Buscar libros con palabras similares
        palabras = nombre_lower.split()
        sugerencias = []
        for libro in libros_disponibles:
            libro_lower = libro.lower()
            if any(palabra in libro_lower for palabra in palabras):
                sugerencias.append(libro)
        
        for i, libro in enumerate(sugerencias[:5], 1):
            print(f"   {i}. {libro}")
        
        return None

# Prueba de la función
print("Prueba de búsqueda:")
prueba = buscar_libro("odyssey", libros_disponibles)


Prueba de búsqueda:
Libro encontrado: The_Odyssey_by_Homer.txt


In [5]:
def recomendar_libros(libro_favorito, n=5, libros_disponibles=None, rdd_sim=None):
    """
    Recomienda N libros más similares al libro favorito del usuario.
    
    Parámetros:
    - libro_favorito: nombre del libro (str)
    - n: cantidad de libros a recomendar (int, default=5)
    - libros_disponibles: lista de libros (list)
    - rdd_sim: RDD con similitudes (RDD)
    
    Retorna:
    - Lista de tuplas: [(libro, similitud), ...]
    - None si el libro no existe
    """
    # Validar que el libro existe
    libro_completo = buscar_libro(libro_favorito, libros_disponibles)
    
    if libro_completo is None:
        return None
    
    print(f"\n Buscando libros similares a: {libro_completo}")
    print(f"   Cantidad solicitada: {n} recomendaciones\n")
    
    # Filtrar similitudes donde aparece el libro
    similitudes_libro = rdd_sim.filter(lambda x: x[0] == libro_completo or x[1] == libro_completo)
    
    # Reformatear: (libro_recomendado, similitud)
    recomendaciones = similitudes_libro.map(
        lambda x: (x[1], x[2]) if x[0] == libro_completo else (x[0], x[2])
    )
    
    # Ordenar por similitud mayor a menor
    top_recomendaciones = recomendaciones.sortBy(lambda x: x[1], ascending=False).take(n)
    
    return top_recomendaciones

In [6]:
# Recomendar libros similares
print("Recomendaciones para el usuario:")

recomendaciones1 = recomendar_libros(
    libro_favorito="The_Odyssey_by_Homer",
    n=5,
    libros_disponibles=libros_disponibles,
    rdd_sim=rdd_sim_parsed
)

if recomendaciones1:
    print("Recomendaciones:")
    for i, (libro, similitud) in enumerate(recomendaciones1, 1):
        print(f"   {i}. [{similitud:.4f}] {libro}")


Recomendaciones para el usuario:
Libro encontrado: The_Odyssey_by_Homer.txt

 Buscando libros similares a: The_Odyssey_by_Homer.txt
   Cantidad solicitada: 5 recomendaciones

Recomendaciones:
   1. [0.1855] The_Iliad_by_Homer.txt
   2. [0.0676] The_Aeneid_by_Virgil.txt
   3. [0.0481] Paradise_Lost_by_John_Milton.txt
   4. [0.0448] The_Complete_Works_of_William_Shakespeare_by_William_Shakespeare.txt
   5. [0.0446] The_Interesting_Narrative_of_the_Life_of_Olaudah_Equiano,_Or_Gustavus_Vassa,_The_African_by_Equiano.txt


In [7]:
# Libro que NO existe
recomendaciones2 = recomendar_libros(
    libro_favorito="Harry Potter",
    n=5,
    libros_disponibles=libros_disponibles,
    rdd_sim=rdd_sim_parsed
)

if recomendaciones2 is None:
    print("\n No se pudieron generar recomendaciones")


Error: El libro 'Harry Potter' no existe en la colección.

 Sugerencias (libros similares):

 No se pudieron generar recomendaciones


In [8]:
sc.stop()